# *Week 1, Exercise 1: Introduction to Segmentation*
 
## YOU HAVE TO HAND IN ON BRIGHTSPACE:

 * This notebook file 
 * The answers to the questions in a separate PDF.
 
### During this practical session we will refresh the following topics:

 * Basic image processing techniques (filtering, morphological operations, etc.)
 * Several types of image thresholing for object segmentation
 * Segmentation using the Region Growing method 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import flood_fill, flood
from skimage.morphology import disk, dilation, erosion, reconstruction
from skimage.filters import threshold_otsu, gaussian
from skimage.color import rgb2gray
from skimage.io import imread, imread_collection


# load the data D from the mri folder
!unzip mri.zip
D = imread_collection('mri/*')

# plotting a few slices

plt.rcParams['figure.figsize'] = [12, 8] # bump up the figure size a bit
fig, axs = plt.subplots(1,3)
axs[0].imshow(D[1], cmap='gray')
axs[0].set_title('Slice 2')

axs[1].imshow(D[13], cmap='gray')
axs[1].set_title('Slice 14')

axs[2].imshow(D[24], cmap='gray')
axs[2].set_title('Slice 25')

plt.show()

## Question 1: 

 * What are the minimum and maximum intensity values in D?

 * What is the type (uint8, unit16, boolean, double) of D? 

 * What are the minimum and maximum intensity values for this type of image?

In [ ]:
# Effect of applying Gaussian filter with given sigma 

fig, axs = plt.subplots(1,3)
axs[0].imshow(gaussian(D[13], 2), cmap = 'gray')
axs[0].set_title('Slice 14, $\sigma$ = 2')

axs[1].imshow(gaussian(D[13], 5), cmap = 'gray')
axs[1].set_title('Slice 14, $\sigma$ = 5')

axs[2].imshow(gaussian(D[13], 10), cmap = 'gray')
axs[2].set_title('Slice 14, $\sigma$ = 10')

plt.show()

In [ ]:
# To plot the histogram of intensity values of the original image and the smoothed version.
# .ravel() is used to convert the 2d slice into a 1d array.

fig, axs = plt.subplots(1,2)
axs[0].hist(D[13].ravel(), 40)
axs[0].set_title('Histogram of slice 14')

axs[1].hist(gaussian(D[13], 5).ravel(), 40)
axs[1].set_title('Histogram of blurred slice 14')

plt.show()

In [ ]:
# Normalize/scale the intensity values to [0, 1] range from [0, 255]. .concatenate is used to allow for
# arithmetic operations on the image collection
image = D.concatenate()/255;

# The histograms can also be plotted as:

image_14 = image[13]
image_14_gauss = gaussian(image[13], 5)

fig, axs = plt.subplots(1,2)
axs[0].hist(image_14.ravel(), 40)
axs[0].set_title('Histogram of slice 14')

axs[1].hist(image_14_gauss.ravel(), 40)
axs[1].set_title('Histogram of blurred slice 14')

plt.show()

In [ ]:
# In this case, it is easier to get the 
# actual numerical data for each bin, in this case 40 values. 
# It is customary to assign values that won't be used to '_'

h,_,__ = plt.hist(image_14.ravel(), 40)

In [ ]:
# In order to create a mask that marks/masks some region of interest, 
# a simple thresholding of a blurred image can be used. 
# In this example, we are going to write our own
# code, which will be useful for many other exercises in this course. 

mask = np.zeros(image_14_gauss.shape)  # create all-zero-image of the same size
mask[image_14_gauss > 0.2] = 1

fig, axs = plt.subplots(1,2)
axs[0].imshow(image_14_gauss, cmap = 'gray')
axs[0].set_title('Original Image')

axs[1].imshow(mask, cmap = 'gray')
axs[1].set_title('Mask')

plt.show()

In [ ]:
# Two other useful image processing operations come from the field of
# mathematical morphology: dilation and erosion. Both of them require a
# structuring element 

se = disk(1)

mask_dil = dilation(mask, se)
mask_ero = erosion(mask, se)

fig, axs = plt.subplots(1,3)
axs[0].imshow(mask, cmap='gray')
axs[0].set_title('Mask')

axs[1].imshow(mask_dil, cmap='gray')
axs[1].set_title('Dilation')

axs[2].imshow(mask_ero, cmap='gray')
axs[2].set_title('Erosion')

plt.show()

In [ ]:
# We can use the morphological operations to find object/region boundaries, for
# example:

fig, axs = plt.subplots(1,2)
axs[0].imshow(mask - mask_ero, cmap='gray')
axs[0].set_title('"Inner" boundary')

axs[1].imshow(mask_dil - mask, cmap='gray')
axs[1].set_title('"Outer" boundary')

plt.show()

In [ ]:
# We can also fill-in some parts of the image in a 'flood-fill' way,
# providing the starting point. The more advanced methods, e.g. region
# growing, will be considered later.
# The parameter 'connectivity' is set to 1 to avoid leakage over diagonally
# connected pixels

fill_im = (mask - mask_ero)

fig, axs = plt.subplots(1,2)
axs[0].imshow(flood_fill(fill_im, (3,3), 1, connectivity = 1), cmap = 'gray')
axs[0].set_title('Filling from (3, 3)')

axs[1].imshow(flood_fill(fill_im, (80,85), 1, connectivity = 1), cmap='gray')
axs[1].set_title('Filling from (80, 85)')

plt.show()

In [ ]:
# A useful function in this course is application of a user-specified 
# lambda function to all the values in an array. Having that, we do
# not really need to use any 'for-loops'. Using the above mentioned function
# one can solve the majority, if not all exercises in this course. Note: all the 
# exercises in this course can be solved without 'for-loops' which make the 
# code more difficult to read and prone to errors. 
# Syntax: (lambda x: 'function of x')(x)
# For example, to invert a uint8 image, we can just change all the
# intensity values according to the function y = 255 - x, 

fig, axs = plt.subplots(1,2)

image_invert = (lambda x: 255 - x)(D[13])
# equivalent to image_invert = 255 - D[13]

axs[0].imshow(image_invert, cmap = 'gray')
axs[0].set_title('Inverted with lambda function')

# Lambda functions can also contain conditional statements (for example:(x > 3)).
# These will return 'True' or 'False' which can be converted to 1 or 0 by
# multiplying by 1. This can be used to avoid 'if-then' statements.

axs[1].imshow((lambda x: (x > 80) * 1 + (x <= 80)*0)(D[13]), cmap='gray')
axs[1].set_title('Thresholded with lambda function')

plt.show()

# Exercise 1.1: Thresholding
 
 * Randomly pick up a slice from D in the range of [0, 26]
 

 * Compute the average intensity value, $\mu$, for that slice and the standard deviation, $\sigma$, excluding all the zero-valued pixels. 
 

 * Display the selected slice and another image in which all the values above $\mu+\sigma$ have values 1, all the pixels with intensities smaller than $\mu - \sigma$ have value 0, and inbetween - value 0.5. The computed $\mu$ and $\sigma$ should be displayed in the title. To display the numbers use an f-string:
 ```f"Normal words {variable}"```.
 
Note: numpy arrays remain linked, use .copy() to avoid changing the original array (f.e. ```slice_n = D[:,:,0,n].copy()```).

In [ ]:
# Excercise 1.1 answer



# Exercise 1.2: Brain segmentation

In this exercise you will determine the fraction of gray matter, white
matter and CSF in a T1-weighted brain image. 

 * First, search for a skull-stripped T1-weighted axial brain image on Google. It is important that the skull is removed, otherwise the segmentation will become more complicated. 
 

 * Using thresholding, segment the image into gray matter, white matter and CSF and determine their relative fractions with respect to the total brain volume (i.e. the three segments together should sum up to 100%). 
 

 * Plot the three segments and put the relative fractions of these segments in the figure titles. 
 
 
 Hint 1: to compute the area of a region/segment in a binary image use numpy's ```np.sum()```. 
 
 Hint 2: If you want to use multiple thresholds at once, use the ```&``` sign: ```BW = (I < thresh1) & (I > thresh2) ```. 

In [ ]:
# Excercise 1.2 answer



In [ ]:
## Region Growing 

#  In this exercise we will use the flood fill function to segment the 
#  brain ventricles on a Magnetic Resonance Image.
 
#  1) Load the brain.png file with the imread function as float data type.

#  2) We transform our rgb image to grayscale with rgb2gray.

#  3) Display the image.

#  4) As we display the image we can select one pixel that represents the 
#  area we want to segment(ventricles). Select one pixel on the left side.

#  5) This pixel coordinates will represent the seed, which is the location 
#  where the algorithm will start growing. The difference between a pixel's intensity 
#  value and the region's mean, is used as a measure of similarity. The pixel
#  with the smallest difference measured this way is allocated to the respective region.
#  This process stops when the intensity difference between region mean and
#  new pixel become larger than a certain threshold. We will use 0.3
#
#  6) We Use the coordinates as input to the region growing function. 
 
image = np.asarray(imread('brain.png'), dtype = float)
image = rgb2gray(image)

image_left_highlighted = flood_fill(image, (124,120), 255, connectivity = 2, tolerance = 0.3*255)

fig, axs = plt.subplots(1,3)
axs[0].imshow(image, cmap = 'gray')
axs[0].set_title(f"Original Image")

axs[1].imshow(image_left_highlighted, cmap = 'gray')
axs[1].set_title(f"Left Ventricle Highlighted")

# 9) We are interested in segmenting the right ventricle as well,
# so repeat the previous steps (5,7 and 8) using a coordinate from the right 
# ventricle. Furthermore, save the result of the region growing algorithm  in  
# variable called |J2| and the addition result on a variable called |z2|.

image_both_highlighted = flood_fill(image_left_highlighted, (124,135), 255, connectivity = 2, tolerance = 0.3*255)

axs[2].imshow(image_both_highlighted, cmap = 'gray')
axs[2].set_title(f"Both Ventricles Highlighted")

plt.show()

## Question 2: 

Explain why the region growing doesn't expand to the right ventricle.


# Exercise 1.3: Liver segmentation

 Segment the liver from the abdomen-CT.jpg using the region growing algorithm.
 
 * After loading the image you will notice that is in rgb. Change it to gray-scale values using the rgb2gray function.
 

 * Select a proper coordinate and threshold for the seed start growing.
 
 
Hint: the tolerance value is a number $< 25$, probably you will need to try
 several values.

* It is possible that your mask of the liver will have many noise and artifacts, you can fix it using a proper morphological operation. 

Hint: the function 'flood' returns the filled region without the original image


* To show the overlay of your liver, add the liver mask to the gray-scale image.


* Plot in one figure the gray-scale image, the mask and the fusion of these two.


As you will find, a perfect segmentation of the liver is not possible with only regiongrowing.


In [ ]:
# Excercise 1.3 answer



## Question 3:

 * According to the number of threshold/ seed coordinate values you had to try, can you mention one/two disadvantages of this segmentation method? 


 * Can you also mention one/two advantages?


 * According to you, how this method  will perform  on organs with
     similar  structure that are  spacely close to each other? 

## Exercise 1.4: Metastases segmentation.

The purpose of this exercise is to segment metastases in the liver and obtain the size of the largest metastasis. To this end, you will need to segment two structures: 

1) a segmentation the liver

2) a segmentation of the metastases

 * Load the image 'livermetastases.jpg'. Do not forget to convert it to a grayscale, double type.
 * Crop the image to the part $[200:399]; [200:399]$.
 * Make a mask for the liver, you can use tresholding and/or region growing
 * Make a mask for the metastases: use morphological operations to
 optimize the result. Note that there is not a single 'correct' solution, 
 try to find a series of operations which gives you a good final
 segmentation. Create an image after every step and output the results as shown below. Please also 
 explain why you decided to use these steps. 
 * Determine the size of the largest metastasis. Assume the voxel size is $0.4*0.4*3$ mm$^3$

In [ ]:
# Excercise 1.4 answer  

